## Qual o tempo de uso das bombas em horário de ponta e fora de ponta?

In [1]:
from pathlib import Path
import pandas as pd 

In [2]:
df_path = Path("../data/curated_data/water_consumption_curated_1.parquet")
df = pd.read_parquet(df_path)
df.head()

,id,timestamp,flow_in_(l/s),reservoir_level_(%),pressure_(mca),gmb_1_is_on,gmb_2_is_on,reservoir_level_liters,time_passed_seconds,liters_should_have_entered,liters_entered,liters_out,flow_out_(l/s)
0,0,2023-03-17 11:27:06,68.59,29.86,38.20,0,1,298600.0,NaN,NaN,NaN,NaN,NaN
1,1,2023-03-17 12:28:56,66.05,35.86,38.20,0,1,358600.0,3710.0,245045.5,60000.0,185045.5,49.88
2,2,2023-03-17 12:31:26,65.64,36.16,38.06,0,1,361600.0,150.0,9846.0,3000.0,6846.0,45.64
3,3,2023-03-17 12:33:56,65.64,36.50,38.03,0,1,365000.0,150.0,9846.0,3400.0,6446.0,42.97
4,4,2023-03-17 12:36:26,65.64,36.80,38.17,0,1,368000.0,150.0,9846.0,3000.0,6846.0,45.64


In [3]:
# Create date and hour columns
df["date"] = df["timestamp"].dt.date
df["hour"] = df["timestamp"].dt.hour

# Create peak and off-peak column
peak_hours = (df["hour"] >= 18) & (df["hour"] <= 21)
df["is_peak_hour"] = peak_hours

# drop time_passed_seconds null values
df = df.dropna(subset=["time_passed_seconds"])

# Calculate water bombs duration 
df['gmb_1_duration'] = df['gmb_1_is_on'] * df['time_passed_seconds']
df['gmb_2_duration'] = df['gmb_2_is_on'] * df['time_passed_seconds']

In [4]:
# Sum water bombs usage time per day and hour 
daily_peak_usage = df[df['is_peak_hour']].groupby('date').agg({'gmb_1_duration': 'sum', 'gmb_2_duration': 'sum'})
daily_off_peak_usage = df[~df['is_peak_hour']].groupby('date').agg({'gmb_1_duration': 'sum', 'gmb_2_duration': 'sum'})

# Calculate water bombs average usage time per day and hour
gmb_1_peak_avg = daily_peak_usage['gmb_1_duration'].mean() / 3600 # convert seconds to hours
gmb_1_off_peak_avg = daily_off_peak_usage['gmb_1_duration'].mean() / 3600 # convert seconds to hours

gmb_2_peak_avg = daily_peak_usage['gmb_2_duration'].mean() / 3600  # convert seconds to hours
gmb_2_off_peak_avg = daily_off_peak_usage['gmb_2_duration'].mean() / 3600  # convert seconds to hours

In [5]:
print(f"Tempo médio de uso da bomba 1 em horário de ponta: {gmb_1_peak_avg:.2f} horas")
print(f"Tempo médio de uso da bomba 1 em horário fora de ponta: {gmb_1_off_peak_avg:.2f} horas")

print(f"Tempo médio de uso da bomba 2 em horário de ponta: {gmb_2_peak_avg:.2f} horas")
print(f"Tempo médio de uso da bomba 2 em horário fora de ponta: {gmb_2_off_peak_avg:.2f} horas")

Tempo médio de uso da bomba 1 em horário de ponta: 1.41 horas
Tempo médio de uso da bomba 1 em horário fora de ponta: 8.00 horas
Tempo médio de uso da bomba 2 em horário de ponta: 0.76 horas
Tempo médio de uso da bomba 2 em horário fora de ponta: 4.44 horas
